In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:5])

gsk_u


### 문제 1-1 : 기본 Chain 만들기 - AI 요리사

In [14]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

template_text = """
너는 재료 기반 요리 추천을 해주는 AI 요리사이다.

[입력 재료]
{ingredients}

[지침]
- 제공된 재료만으로 만들 수 있는 한국 요리 1가지를 추천해.

[출력 형식]
입력: {ingredients}
추천 요리: <요리명>
레시피:
1) ...
2) ...
3) ...
4) ...
"""
prompt = PromptTemplate.from_template(template_text)

llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",   
    model="moonshotai/kimi-k2-instruct-0905",      
    temperature=0.4                                  
)

chain = prompt | llm | StrOutputParser()
ingredients = "갈비, 무"
response = chain.invoke({"ingredients": ingredients})
print(response)


입력: 갈비, 무  
추천 요리: 무갈비찜  
레시피:  
1) 갈비는 흐르는 물에 핏물을 제거하고, 냄비에 넣어 찬물을 붓고 끓어오르면 5분 정도 삶아 거품을 걷어낸다.  
2) 무는 3 cm 두께로 썰어 큰 조각으로 준비한다.  
3) 삶은 갈비와 무를 냄비에 담고, 물을 재료를 살짝 덮을 만큼 붓는다.  
4) 센 불에서 끓이다가 물이 반으로 줄면 중불로 줄이고, 30분간 더 끓여 무가 투명해지고 갈비가 연해지면 완성.


### 문제 1-2 : 2단계 Multi Chain 만들기 - 여행지 정보 시스템


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from operator import itemgetter
import re

llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0.3,
)

stage1_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",
         "너는 여행 플래너다. 사용자가 준 지역과 관심사를 바탕으로 대표 장소 1곳만 추천한다. "
         "한국어로 간결히 답하고 출력 형식을 지켜라."),
        ("user",
         "지역: {region}\n관심사: {theme}\n\n"
         "[출력 형식]\n추천 장소: <장소명>\n한줄 이유: <40자 이내>")
    ]
)
stage1_chain = stage1_prompt | llm | StrOutputParser()

def extract_place(text: str) -> str:
    m = re.search(r"추천\s*장소\s*:\s*(.+)", text)
    if m:
        return m.group(1).strip()
    first = text.strip().splitlines()[0]
    return first.split(":", 1)[1].strip() if ":" in first else first.strip()

stage2_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",
         "너는 여행 가이드다. 주어진 장소에 대해 간결하고 사실 위주로 설명한다. "
         "각 항목은 2~3문장, 불릿 리스트를 사용한다."),
        ("user",
         "지역: {region}\n관심사: {theme}\n장소: {place}\n\n"
         "[요청]\n- 간단 소개(1문장)\n- 역사(• 2~3문장)\n- 특징(• 2~3문장)\n- 방문 팁(• 2~3문장)\n\n"
         "[출력 형식]\n장소: {place}\n소개: <1문장>\n역사:\n- ...\n특징:\n- ...\n방문 팁:\n- ...")
    ]
)
stage2_chain = stage2_prompt | llm | StrOutputParser()

make_stage1_bundle = (
    {
        "region": itemgetter("region"),
        "theme": itemgetter("theme"),
        "stage1_raw": stage1_chain,
    }
)

add_place = RunnableLambda(
    lambda x: {**x, "place": extract_place(x["stage1_raw"])}
)

run_stage2 = (
    RunnableLambda(lambda x: {"region": x["region"], "theme": x["theme"], "place": x["place"]})
    | stage2_chain
)

final_format = RunnableLambda(
    lambda x: (
        "=== 1단계 결과 ===\n"
        f"{x['stage1_raw']}\n\n"
        "=== 2단계 결과 ===\n"
        f"{x['stage2_detail']}\n"
    )
)

full_chain = (
    make_stage1_bundle
    | add_place
    | {
        "region": itemgetter("region"),
        "theme": itemgetter("theme"),
        "place": itemgetter("place"),
        "stage1_raw": itemgetter("stage1_raw"),
        "stage2_detail": run_stage2, 
    }
    | final_format
)

# ===== 실행 예시 =====
if __name__ == "__main__":
    print(full_chain.invoke({"region": "로마", "theme": "역사"}))
    print(full_chain.invoke({"region": "도쿄", "theme": "미식"}))


=== 1단계 결과 ===
추천 장소: 콜로세움
한줄 이유: 로마 제국의 상징, 검투사들의 피가 느껴지는 2천 년 역사의 현장

=== 2단계 결과 ===
장소: 콜로세움  
소개: 로마의 대표적인 원형 경기장으로 고대 로마 제국의 권위와 엔터테인먼트 문화를 상징한다.

역사:  
- 서기 72년 황제 베스파시아누스가 착공해 80년 아들 티투스에 의해 완공되었다.  
- 5만 명을 수용한 이 원형 경기장에서는 검투사 대전, 야수 사냥, 해상 전투 재현 등이 400년 이상 열렸다.  
- 중세 이후 요새·채석장으로도 쓰였으며 18세기부터 보존·발굴이 본격화되었다.

특징:  
- 총 높이 48m, 둘레 527m의 4층 석재 원형 구조는 반원 아치 기술의 집약체다.  
- 무대 아래 하이포지움(지하 통로)는 검투사와 야수를 올리던 승강 장치·감옥·수조를 보유했다.  
- 외벽 3층까지는 원형 아치, 4층은 직사각형 창이 특징이며 콘크리트·석회·벽돌이 혼용되었다.

방문 팁:  
- 온라인 예약 없이는 현장 표 매입이 어려우므로 최소 하루 전에 결제 완료한다.  
- 오전 8:30 개장 직후나 해질노을이 떨어지는 1시간 전이 사진 명소다.  
- 로마 유적 통합권(콜로세움·포로 로마노·팔라티노)을 구매하면 24시간 내 1회 재입장이 가능하다.

=== 1단계 결과 ===
추천 장소: 메트로폴리탄 미술관
한줄 이유: 5천년 세계 문명의 명품들이 한곳에, 뉴욕 문화의 심장

=== 2단계 결과 ===
장소: 메트로폴리탄 미술관
소개: 세계 3대 미술관 중 하나로 5천년간의 인류 예술품 200만 점을 소장한 뉴욕의 문화 아이콘이다.
역사:
- 1870년 뉴욕 상업가·예술가들이 ‘미술을 미국인에게’라는 취지로 설립했으며, 초기 건물은 도서관 개조였다.
- 1880년대 중앙공원 옆 현재 위치에 고딕·네오클래식 양식의 본관을 완공하고 150년간 5차 확장을 거쳐 오늘날 규모를 갖췄다.
특징:
- 이집트·근동·유럽 회화·무기·의상 등 17개 부서가 2층~2층 반 규모로 펼쳐져 

### 문제 1-3 : FewShotPromptTemplate과 시스템 메시지 활용 

In [ ]:
# 문제 1-3: FewShotPromptTemplate과 시스템 메시지 활용 - 뉴스 키워드 추출기

from langchain_core.prompts import FewShotChatMessagePromptTemplate, ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
import re

llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0.2, 
)

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{news}"),
    ("ai", "키워드: {keywords}")
])

examples = [
    {  # IT
        "news": "삼성전자가 내년 초 자체 인공지능(AI) 가속기를 출시해 엔비디아 독점에 도전한다.",
        "keywords": "삼성전자, 인공지능, 엔비디아"
    },
    {  # 보건
        "news": "세계보건기구(WHO)는 글로벌 보건 시스템 강화를 위해 국제 협력의 필요성을 재차 강조했다.",
        "keywords": "세계보건기구, 국제협력, 보건시스템"
    },
    {  # 경제
        "news": "한국은행이 기준금리를 동결하며 물가 안정과 경기 둔화 사이 균형을 강조했다.",
        "keywords": "한국은행, 기준금리, 물가"
    },
    {  # 스포츠
        "news": "손흥민이 프리미어리그 경기에서 멀티골을 기록하며 팀 승리를 이끌었다.",
        "keywords": "손흥민, 프리미어리그, 멀티골"
    },
    {  # 정치/외교
        "news": "한미 정상은 정상회담에서 첨단 기술 동맹과 공급망 협력을 확대하기로 합의했다.",
        "keywords": "정상회담, 기술동맹, 공급망"
    },
]

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "너는 뉴스 키워드 추출 전문가이다. 아래 규칙을 반드시 지켜라.\n"
     "- 핵심 키워드 정확히 3개만 추출\n"
     "- 각 키워드는 1~3단어 내의 고유명사/핵심명사 위주(불용어, 조사 제외)\n"
     "- 해시태그/따옴표/불필요 문구 없이 쉼표로 구분\n"
     "- 최종 출력 형식: '키워드: A, B, C'"),
    few_shot_prompt,
    ("human", "{input}")
])


def normalize_keywords(text: str) -> str:
  
    m = re.search(r"키워드\s*:\s*(.*)", text, flags=re.IGNORECASE | re.DOTALL)
    body = m.group(1).strip() if m else text.strip()

    parts = re.split(r"[,\n/·|]+", body)
 
    cleaned = []
    for p in parts:
        k = re.sub(r"[\[\]\(\)\"'`#]", "", p).strip()
        if k:
            cleaned.append(k)
    
    cleaned = cleaned[:3] if len(cleaned) >= 3 else cleaned + (["키워드부족"] * (3-len(cleaned)))
    return f"키워드: {cleaned[0]}, {cleaned[1]}, {cleaned[2]}"

normalizer = RunnableLambda(normalize_keywords)


chain = final_prompt | llm | StrOutputParser() | normalizer

test_news = (
    "제미나이 2.0 플래시는 현재 구글 AI 스튜디오(Google AI Studio) 및 버텍스 AI(Vertex AI)에서 "
    "제미나이 API를 통해 개발자에게 실험 모델로 제공됩니다. 모든 개발자는 멀티모달 입력 및 텍스트 출력을 사용할 수 있으며, "
    "텍스트 음성 변환(text-to-speech) 및 네이티브 이미지 생성은 일부 파트너들을 대상으로 제공됩니다. "
    "내년 1월에는 더 많은 모델 사이즈와 함께 일반에 공개될 예정입니다."
)

print(chain.invoke({"input": test_news}))


키워드: 제미나이 2.0, 구글 AI 스튜디오, 멀티모달
